In [1]:
import gymnasium as gym
import ale_py

# Register the environments
gym.register_envs(ale_py)

# Create and test an environment
env = gym.make('ALE/Breakout-v5', render_mode="human")
obs, _ = env.reset()
print("Environment created successfully!")
print(f"Observation shape: {obs.shape}")

for _ in range(100):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, _ = env.step(action)
    if terminated or truncated:
        obs, _ = env.reset()

env.close()

Environment created successfully!
Observation shape: (210, 160, 3)
